# Parte 5

En este ejercicio se predecirá si un paciente presenta una enfermedad cardiaca a partir de ciertos indicadores mediante una regresión lineal. La variable a predecir será el máximo ritmo cardíaco alcanzado y se comprobará que tan distante es el valor real con el predecido, para así poder clasificar el paciente como que posee una enfermedad o no.

   ## a) Carga de datos

Se realiza la lectura del archivo de datos y se divide el dataframe en dos, un dataframe de entrenamiento con el 70% de los datos y uno de prueba con el restante 30%.

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import sklearn.linear_model as lm
import numpy as np

headers = ['age','sex','chest_pain','blood_p','serum',
'blood_s','electro','max_heart','angina','oldpeak','slope','vessel','thal','normal']
df = pd.read_csv('heart.dat', header=None, names=headers, sep=' ')
print"Forma:",df.shape
df.info()
df.describe()
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
X = df_scaled.iloc[:,1:]
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['max_heart']
mascara = np.zeros(len(X))
limit = int(len(X)*0.7)
mascara[:limit] = 1
istrain = mascara== 1
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
print "Cantidad de datos de entrenamiento:", len(Xtrain) 
print "Cantidad de datos de prueba:",len(Xtest)


Forma: (270, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 14 columns):
age           270 non-null float64
sex           270 non-null float64
chest_pain    270 non-null float64
blood_p       270 non-null float64
serum         270 non-null float64
blood_s       270 non-null float64
electro       270 non-null float64
max_heart     270 non-null float64
angina        270 non-null float64
oldpeak       270 non-null float64
slope         270 non-null float64
vessel        270 non-null float64
thal          270 non-null float64
normal        270 non-null int64
dtypes: float64(13), int64(1)
memory usage: 29.6 KB
Cantidad de datos de entrenamiento: 189
Cantidad de datos de prueba: 81


Aplicamos nuevamente los métodos .info() , describe() y shape() para analizar los datos entregados. Observamos que se trata de 270 registros, cada uno con 14 atributos relevantes para la predicción, como es la edad, la presión arterial, el colesterol entre otros. En cuanto al tipo de datos, predominan los atributos con cantidades flotantes, excepto el atributo *normal* el cual es un numérico que simboliza la presencia**(2)** o no presencia**(1)** de enfermedad.

### b) Regresión lineal y umbral

A continuación se procede a realizar la regresión lineal y se determina un umbral para determinar si un paciente presenta o no una enfermedad cardíaca.

In [4]:
import numpy.linalg as LA
linreg = lm.LinearRegression(fit_intercept = False)
modelo = linreg.fit(Xtrain, ytrain)
yhat_val = modelo.predict(Xtest)
error_test = np.mean(np.power(yhat_val - ytest, 2))
print "Error de test:",error_test 

from sklearn.metrics import accuracy_score

ytest_arr = np.array(ytest)

residuos = []
for i in range(0,81):
    residuos.append(yhat_val[i]- ytest_arr[i])  #Valor predecido - valor real

    
promedio_dif = np.mean(residuos)
desviacion_dif = np.std(residuos)
print "Promedio:",promedio_dif
print "desviacion:",desviacion_dif


enfermos_prediccion = np.ones(len(ytest))

#Y_test
promedio = np.mean(ytest)
maximo = np.max(ytest)
minimo = np.min(ytest)
std = np.std(ytest)
X_n = df.iloc[:,1:]
N = X_n.shape[0]
X_n.insert(X_n.shape[1], 'intercept', np.ones(N))
y = df['max_heart']
mascara = np.zeros(len(X_n))
limit = int(len(X_n)*0.7)
mascara[:limit] = 1
istrain = mascara== 1
Xtrain = X_n[istrain]
Xtest = X_n[np.logical_not(istrain)]
enfermos_reales = list(Xtest[:]['normal'])


for i in range(len(ytest)):
    if(ytest_arr[i] > promedio + 2*std or ytest_arr[i] < promedio - 2*std):
        enfermos_prediccion[i] = 2
    else:
        enfermos_prediccion[i] = 1


print "Accuracy score 2 desviaciones:", (accuracy_score(enfermos_reales,enfermos_prediccion))

## 2 desviaciones fue un umbral muy crítico, con 1 desviacion :

for i in range(len(ytest)):
    if(ytest_arr[i] > promedio + std or ytest_arr[i] < promedio -std):
        enfermos_prediccion[i] = 2
    else:
        enfermos_prediccion[i] = 1

        

for i in range(len(ytest)):
    if(ytest_arr[i] > promedio  or ytest_arr[i] < promedio ):
        enfermos_prediccion[i] = 2
    else:
        enfermos_prediccion[i] = 1
        
        
        
for i in range(len(ytest)):
    if(ytest_arr[i] > promedio + 3*std or ytest_arr[i] < promedio - 3*std):
        enfermos_prediccion[i] = 2
    else:
        enfermos_prediccion[i] = 1
        
print "Accuracy 3 desviación:", (accuracy_score(enfermos_reales,enfermos_prediccion))



Error de test: 2.81867293711e-27
Promedio: -5.20756705933e-14
desviacion: 1.03342861082e-14
Score 2 desviaciones: 0.592592592593
Score 3 desviación: 0.567901234568


La frontera de decisión adoptada, estuvo basada en calcular el promedio de las diferencias entre el valor predecido y real de el máximo ritmo cardiaco con 2 o 3 desviaciones estándar, entonces se recorren las predicciones y se van clasificiaciones según el criterio anteriormente mencionado. Si el valor predecido supera el umbral será considerado como un paciente enfermo.
El desempeño de este criterio no es tan alto como uno esperaría, con 2 desviaciones se obtiene un $59\%$ y con 3 $56\%$. De esto se puede deducir que esta aproximación no es la óptima ya que para clasificar se está usando una regresión lineal como base, podría ser mejor si directamente al modelo se le implementa una función de pérdida correspondiente a un problema de clasificación.